In [45]:
import numpy as np
import os

import torch
import torch.utils
import torch.distributions
import torchvision
import tqdm

from diffusers.models import AutoencoderKL

In [46]:
IMAGE_FOLDER = 'data/faces4/256x256/faces'
LATENT_IMAGE_FOLDER = 'data/latent_faces4'
TARGET_SIZE = 256

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def init_vae(device):
    # https://huggingface.co/stabilityai/sd-vae-ft-mse
    model: AutoencoderKL = AutoencoderKL.from_pretrained('stabilityai/sd-vae-ft-mse').to(device)
    torch.compile
    model = model.eval()
    model.train = False
    for param in model.parameters():
        param.requires_grad = False
    return model

vae = init_vae(device)
scale_factor=0.18215 # scale_factor follows DiT and stable diffusion.

@torch.no_grad()
def encode(x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]: 
    posterior = vae.encode(x, return_dict=False)[0].parameters
    return torch.chunk(posterior, 2, dim=1)    

@torch.no_grad()
def sample(mean: torch.FloatTensor, logvar: torch.FloatTensor) -> torch.FloatTensor:
    std = torch.exp(0.5 * logvar)
    z = torch.randn_like(mean)
    z = mean + z * std
    return z * scale_factor

@torch.no_grad()
def decode(z) -> torch.Tensor:
    x = vae.decode(z / scale_factor, return_dict=False)[0]
    x = ((x + 1.0) * 127.5).clamp(0, 255).to(torch.uint8)
    return x

In [48]:

def list_files_recursively(data_dir):
    results = []
    for entry in sorted(os.listdir(data_dir)):
        full_path = os.path.join(data_dir, entry)
        ext = os.path.splitext(full_path)[1].lower()
        if os.path.isdir(full_path):
           results.extend(list_files_recursively(full_path))
        elif ext in ['.jpg', '.jpeg', '.png', '.bmp']:
            results.append(full_path) 
    return results

def center_crop_square(image: torch.Tensor):
    # Crop center of image to be square using pytorch
    _, width, height = image.shape
    new_size = min(width, height)

    left = (width - new_size) // 2
    top = (height - new_size) // 2
    right = (width + new_size) // 2
    bottom = (height + new_size) // 2

    return image[:, left:right, top:bottom]

def resize_square(image: torch.Tensor, size: int): 
  return torchvision.transforms.functional.resize(image, size, interpolation=torchvision.transforms.InterpolationMode.BICUBIC)

image_files = list_files_recursively(IMAGE_FOLDER)

# For each image, load it, resize it, and save it   

for image_file in tqdm.tqdm(image_files):
  # Load image RGB

  image = torchvision.io.read_image(image_file, mode=torchvision.io.image.ImageReadMode.RGB)
  image = image.to(device=device, dtype=torch.float32)

  # Preprocess

  image = center_crop_square(image)
  image = resize_square(image, TARGET_SIZE)
  image = (image / 127.5) - 1
  image = image.unsqueeze(0)  

  # Encode

  with torch.cuda.amp.autocast(): # Imrpoves performance and reduces memory footprint on NVIDIA GPUs
    mean, logvar = encode(image)

  # Save

  mean = mean.cpu().numpy().astype(np.float16)
  logvar = logvar.cpu().numpy().astype(np.float16)

  npz_file = os.path.splitext(image_file.replace(IMAGE_FOLDER, LATENT_IMAGE_FOLDER))[0] + '.npz'

  np.savez_compressed(npz_file, mean=mean, logvar=logvar)

  # Load

  # loaded = np.load(npz_file)

  # mean = loaded['mean']
  # logvar = loaded['logvar']

  # mean = torch.tensor(mean).to(device).to(torch.float32)
  # logvar = torch.tensor(logvar).to(device).to(torch.float32)

  # # Decode

  # z = sample(mean, logvar)
  # x = decode(z).cpu()
  # x = x.squeeze(0)

  # # Save

  # torchvision.io.write_jpeg(x, image_file.replace(IMAGE_FOLDER, LATENT_IMAGE_FOLDER), quality=100)


100%|██████████| 34939/34939 [20:48<00:00, 27.98it/s]
